In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
columns = ['From', 'To', 'Conversation_Id', 'Date', 'Timestamp', 'Message', 'Time']
data = pd.read_csv("/home/anku5h/Downloads/sms-demo-data.csv",sep='&&',names = columns)

/home/anku5h/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [3]:
data.head(10)

,From,To,Conversation_Id,Date,Timestamp,Message,Time
0,56161940,ME,1,2-Feb-2018,1517567920165,Use 64810 to verify t4sfmei@cunningsid-bear-20...,16:08:40
1,"""IM-MAXFSN",ME,2,15-Sep-2018,1536984019841,You picked the best of MAX's men's wear collec...,"09:30:19"""
2,Nick,ME,3,15-Oct-2018,1539611987271,Hey,19:29:47
3,ME,Nick,4,15-Oct-2018,1539612422034,Wassup nigga,19:37:02
4,Nick,ME,3,30-Oct-2018,1540881657121,Hungry??,12:10:57
5,ME,Nick,4,30-Oct-2018,1540881828850,No,12:13:48
6,Nick,ME,3,30-Oct-2018,1540882820352,Wanna eat?,12:30:20
7,ME,Nick,4,30-Oct-2018,1540882858010,Sure,12:30:58
8,ME,Pal,4,16-May-2019,1557990530386,Wassup yo,12:38:50
9,Pal,ME,3,16-May-2019,1557990877583,Nothing yo just chilling...,12:44:37


As you can see there's no way to get all conversations with some person, as the 'ME' has a seperate cluster(Conversation_Id). Here for all contacts the 'ME' conversation_id remains the same.
So lets create a new cluster with all matching ME values to whom having conversation with.

In [4]:
# taking unique names from data[from] to assign them a cluster
data['Conversation_Id'] = data['Conversation_Id'].astype(int)
conv_ids = data['Conversation_Id'].values
names = data['From'].unique() # all names from data[from]

In [5]:
#create a instance of a cluster for frequent names from data[to] column
some = data.loc[data['From'] == 'ME','To'] #sent messages by ME, select their names
me_indexs = some.index
some_unique = some.unique()
new_cluster_with_name = {k: v for k, v in (zip(some_unique, np.arange(1,len(me_indexs))))} # cluster with frequent names
rest_values = [] 
for name in names:
    if name not in new_cluster_with_name: # not in freqeunt cluster
        rest_values.append(name) # add to array
rest_of_clusters = {k: v for k, v in (zip(rest_values, np.arange(26,584)))} # new cluster for rest of the names

In [6]:
new_cluster_with_name # freqeunts

{'Nick': 1, 'Pal': 2}

In [7]:
new_cluster_with_name.update(rest_of_clusters) # joining both cluster

In [8]:
new_cluster_with_name

{'Nick': 1,
 'Pal': 2,
 '56161940': 26,
 '"IM-MAXFSN': 27,
 'ME': 28,
 'IM-PLAYIT': 29,
 'BR-JEETLO': 30}

In [9]:
# Assigning all data[from] values a cluster from our new cluster
con_range = np.arange(0,len(data['From']))
for val, name in zip(con_range, data['From'].values):
    if name not in new_cluster_with_name:
        conv_ids[val] = int(new_cluster_with_name['(Unknown)'])
    else:   
        conv_ids[val] = int(new_cluster_with_name[name]) 

In [10]:
# substituiting into a DataFrame
data['Conversation_ID'] = conv_ids

In [11]:
# Extracting me values indexs
some = data.loc[data['From'] == 'ME','To']
me_indexs = some.index

In [12]:
# sorting me values
for val, name in zip(me_indexs, some.values):
    conv_ids[val] = int(new_cluster_with_name[name]) # giving conv_id from our cluster to me indexs value of data[conv]

In [13]:
data['Conversation_ID'] = conv_ids # substituiting

In [14]:
#removing previous cluster(conversation_Id), add a new.
new_data = data[['From','To', 'Conversation_ID', 'Date', 'Timestamp', 'Message' ,'Time']] 

In [15]:
new_data

,From,To,Conversation_ID,Date,Timestamp,Message,Time
0,56161940,ME,26,2-Feb-2018,1517567920165,Use 64810 to verify t4sfmei@cunningsid-bear-20...,16:08:40
1,"""IM-MAXFSN",ME,27,15-Sep-2018,1536984019841,You picked the best of MAX's men's wear collec...,"09:30:19"""
2,Nick,ME,1,15-Oct-2018,1539611987271,Hey,19:29:47
3,ME,Nick,1,15-Oct-2018,1539612422034,Wassup nigga,19:37:02
4,Nick,ME,1,30-Oct-2018,1540881657121,Hungry??,12:10:57
5,ME,Nick,1,30-Oct-2018,1540881828850,No,12:13:48
6,Nick,ME,1,30-Oct-2018,1540882820352,Wanna eat?,12:30:20
7,ME,Nick,1,30-Oct-2018,1540882858010,Sure,12:30:58
8,ME,Pal,2,16-May-2019,1557990530386,Wassup yo,12:38:50
9,Pal,ME,2,16-May-2019,1557990877583,Nothing yo just chilling...,12:44:37


In [16]:
new_data.loc[data['Conversation_ID'] == 1]

,From,To,Conversation_ID,Date,Timestamp,Message,Time
2,Nick,ME,1,15-Oct-2018,1539611987271,Hey,19:29:47
3,ME,Nick,1,15-Oct-2018,1539612422034,Wassup nigga,19:37:02
4,Nick,ME,1,30-Oct-2018,1540881657121,Hungry??,12:10:57
5,ME,Nick,1,30-Oct-2018,1540881828850,No,12:13:48
6,Nick,ME,1,30-Oct-2018,1540882820352,Wanna eat?,12:30:20
7,ME,Nick,1,30-Oct-2018,1540882858010,Sure,12:30:58


In [17]:
new_data.loc[data['Conversation_ID'] == 2]

,From,To,Conversation_ID,Date,Timestamp,Message,Time
8,ME,Pal,2,16-May-2019,1557990530386,Wassup yo,12:38:50
9,Pal,ME,2,16-May-2019,1557990877583,Nothing yo just chilling...,12:44:37
10,ME,Pal,2,16-May-2019,1557990886215,Cool,12:44:46


Sweet!